For Google Colab users, setup the environment and install pypsa. 

In [ ]:
# !pip install pypsa pypsa[excel]

In [ ]:
# from google.colab import drive

# import os

# drive.mount('/content/drive')

# # Define your assignments folder path
# assignments_path = '/content/drive/MyDrive/mec4131z-assignments-2025'

# # Change working directory (optional, only if you want)
# os.chdir(assignments_path)


# Assignment 4 Notebook

Use this notebook for your assignment, there are plotting functions and the export code at the end. 

In [ ]:
import pypsa
import pandas as pd
pd.options.plotting.backend = "plotly"
n = pypsa.Network("assignment-4-case1.xlsx")
# n = pypsa.Network("assignment-4-case2.xlsx")


In [ ]:
n.generators

In [ ]:
n.generators_t.p_max_pu.plot()

In [ ]:
n.generators_t.p_min_pu.plot()

In [ ]:
n.loads_t.p_set.plot()

In [ ]:
n.optimize()

In [ ]:
import plotly.graph_objects as go

# Find generator names containing 'Loadshedding' or 'Overcapacity'
loadshedding_names = n.generators.index[n.generators.index.str.contains('Loadshedding|Overcapacity')]

# Filter the time series
loadshedding_p = n.generators_t.p[loadshedding_names]

# Plot
fig = go.Figure()

for col in loadshedding_p.columns:
    fig.add_trace(go.Scatter(
        x=loadshedding_p.index,
        y=loadshedding_p[col],
        mode='lines',
        name=col
    ))

fig.update_layout(
    title="Loadshedding and Overcapacity Generator Dispatch Over Time",
    xaxis_title="Time",
    yaxis_title="Power Output (MW)",
    legend_title="Generator",
    hovermode="x unified"
)

fig.show()


In [ ]:
n.generators

In [ ]:
n.generators_t.p.describe()

In [ ]:
n.storage_units_t.state_of_charge.plot()

In [ ]:
n.storage_units_t.p.plot()

In [ ]:
import pandas as pd

# Assume your network is called 'network'
# 1. Merge generator power time series with generator attributes
gen_p = n.generators_t.p.copy()
gen_p['carrier'] = n.generators.carrier

# 2. Sum across time
total_energy_per_generator = gen_p.drop(columns="carrier").sum()

# 3. Map generators to carriers
carrier_map = n.generators.carrier
total_energy_per_carrier = total_energy_per_generator.groupby(carrier_map).sum()

print(total_energy_per_carrier)


In [ ]:
import folium
import pandas as pd
import numpy as np

# Assume you already have:
# n.buses with ['x', 'y'] (longitude, latitude)
# n.links with ['bus0', 'bus1', 'p_nom']
# n.links_t.p0 and n.links_t.p1

# 1. Setup a folium Map (centered roughly at South Africa)
m = folium.Map(location=[-29.0, 24.0], zoom_start=5)

# 2. Add dots for each bus
for bus, row in n.buses.iterrows():
    folium.CircleMarker(
        location=[row['y'], row['x']],
        radius=5,
        color='black',
        fill=True,
        fill_opacity=1,
        popup=bus
    ).add_to(m)

# 3. Draw lines between buses
for link, row in n.links.iterrows():
    try:
        # Get bus coordinates
        bus0 = n.buses.loc[row['bus0']]
        bus1 = n.buses.loc[row['bus1']]
        
        # Get total flow (mean absolute flow over time)
        flow0 = n.links_t.p0[link].abs().mean()
        flow1 = n.links_t.p1[link].abs().mean()
        total_flow = flow0 + flow1

        # Get p_nom for coloring
        p_nom = row['p_nom']

        # Choose color based on p_nom (simple scaling)
        if p_nom < 100:
            color = 'green'
        elif p_nom < 300:
            color = 'orange'
        else:
            color = 'red'

        # Line weight proportional to total_flow
        weight = max(1, total_flow / 1000)  # adjust the scale if needed

        folium.PolyLine(
            locations=[[bus0['y'], bus0['x']], [bus1['y'], bus1['x']]],
            color=color,
            weight=weight,
            popup=f"{link}: {total_flow:.1f} MW flow, {p_nom} MW rating"
        ).add_to(m)

    except KeyError:
        # Skip if a bus is missing
        continue

# 4. Show map
m


In [ ]:
n.generators_t.p.plot(kind="area")

In [ ]:
link_loading = n.links_t.p0.div(n.links.p_nom.reindex(n.links_t.p0.columns))
link_loading.plot()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

# Assuming n.links_t.p0 is a DataFrame
p0 = n.links_t.p0

# Calculate how many rows needed (2 columns)
n_links = p0.shape[1]
n_rows = math.ceil(n_links / 2)

# Create subplot figure
fig = make_subplots(rows=n_rows, cols=2, subplot_titles=p0.columns.tolist())

# Loop over each link and add a plot
for i, col in enumerate(p0.columns):
    row = i // 2 + 1
    col_num = i % 2 + 1
    fig.add_trace(
        go.Scatter(x=p0.index, y=p0[col], name=col),
        row=row, col=col_num
    )

# Update layout for readability
fig.update_layout(height=300*n_rows, width=1000, title_text="Link Flows (p0)", showlegend=False)

# Show the figure
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

# Assuming n.links_t.p0 and n.links.p_nom are available
p0 = n.links_t.p0
p_nom = n.links.p_nom

# Normalise p0 by p_nom for each line
p0_normalized = p0.copy()
for col in p0.columns:
    if col in p_nom.index:
        p0_normalized[col] = p0[col] / p_nom[col]
    else:
        p0_normalized[col] = p0[col]  # if p_nom missing, leave as is

# Calculate how many rows needed (2 columns)
n_links = p0_normalized.shape[1]
n_rows = math.ceil(n_links / 2)

# Create subplot figure
fig = make_subplots(rows=n_rows, cols=2, subplot_titles=p0_normalized.columns.tolist())

# Loop over each link and add a plot
for i, col in enumerate(p0_normalized.columns):
    row = i // 2 + 1
    col_num = i % 2 + 1
    fig.add_trace(
        go.Scatter(x=p0_normalized.index, y=p0_normalized[col], name=col),
        row=row, col=col_num
    )

# Update layout for readability
fig.update_layout(height=300*n_rows, width=1000, title_text="Normalized Link Flows (p0 / p_nom)", showlegend=False)

# Show the figure
fig.show()


In [ ]:
n.storage_units_t.p.plot(kind="area")

In [ ]:
import pandas as pd

# Copy the storage p dataframe
storage_p = n.storage_units_t.p.copy()

# Create two new DataFrames for dispatch and charge
storage_dispatch = storage_p.clip(lower=0)  # Keep only positive (discharge)
storage_charge = (storage_p.clip(upper=0))  # Flip sign for charging

# Rename the indices to distinguish
storage_dispatch.index.name = 'snapshot'
storage_dispatch.columns = [f"{col} dispatch" for col in storage_dispatch.columns]

storage_charge.index.name = 'snapshot'
storage_charge.columns = [f"{col} charge" for col in storage_charge.columns]

# Now, bring in the generator p
generator_p = n.generators_t.p.copy()
generator_p.index.name = 'snapshot'

# Concatenate everything
combined_df = pd.concat([generator_p, storage_dispatch, storage_charge], axis=1)

# Optional: sort columns if you want it tidy
combined_df = combined_df.sort_index(axis=1)



In [ ]:
combined_df.plot(kind="area", title="Power Production and Storage Dispatch/Charge")

In [ ]:
combined_df.columns

In [ ]:
plot_order = [
    'EC Overcapacity',
    'FS Overcapacity',
    'GP Overcapacity',
    'ZN Overcapacity',
    'LP Overcapacity',
    'MP Overcapacity',
    'NC Overcapacity',
    'NW Overcapacity',
    'WC Overcapacity',
    'BESS EC charge',
    'BESS FS charge',
    'BESS GP charge',
    'BESS ZN charge',
    'BESS LP charge',
    'BESS MP charge',
    'BESS NW charge',
    'BESS NC charge',
    'BESS WC charge',
    'Drakensberg charge',
    'Ingula charge',
    'Palmiet charge',
    'Steenbras charge',
    'CFPP Mpumalanga',
    'CFPP Free State',
    'CFPP Limpopo',
    'Solar Eastern Cape',
    'Solar Free State',
    'Solar Gauteng',
    'Solar KwaZulu-Natal',
    'Solar Limpopo',
    'Solar Mpumalanga',
    'Solar North West',
    'Solar Northern Cape',
    'Solar Western Cape',
    'Wind Eastern Cape',
    'Wind Free State',
    'Wind Gauteng',
    'Wind KwaZulu-Natal',
    'Wind Limpopo',
    'Wind Mpumalanga',
    'Wind North West',
    'Wind Northern Cape',
    'Wind Western Cape',
    'OCGT Eastern Cape',
    'OCGT Western Cape',
    'CCGT KZN',
    'BESS EC dispatch',
    'BESS FS dispatch',
    'BESS GP dispatch',
    'BESS ZN dispatch',
    'BESS LP dispatch',
    'BESS MP dispatch',
    'BESS NW dispatch',
    'BESS NC dispatch',
    'BESS WC dispatch',
    'Drakensberg dispatch',
    'Ingula dispatch',
    'Palmiet dispatch',
    'Steenbras dispatch',
    'EC Loadshedding',
    'FS Loadshedding',
    'GP Loadshedding',
    'ZN Loadshedding',
    'LP Loadshedding',
    'MP Loadshedding',
    'NC Loadshedding',
    'NW Loadshedding',
    'WC Loadshedding'
]




# Reorder the combined_df
# Drop missing ones first to avoid KeyError if any label doesn't exist
columns_in_df = [col for col in plot_order if col in combined_df.columns]

combined_df = combined_df[columns_in_df]


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

for col in combined_df.columns:
    fig.add_trace(go.Scatter(
        x=combined_df.index,
        y=combined_df[col],
        mode='lines',
        stackgroup='one',
        name=col
    ))

fig.update_layout(
    title="Stacked Area Plot",
    xaxis_title="Time",
    yaxis_title="Power [MW]",
    legend_title="Units",
    hovermode="x unified"
)

fig.show()


In [ ]:
#to export results to excel

# n.export_to_excel("assignment-4-casex-results.xlsx")

In [ ]:
#to export results to ecsv

# n.export_to_csv("assignment-4-casex-results")